In [93]:
import tushare as ts
import time
import pyttsx3
import pandas as pd
from datetime import datetime
import os 

hl_dic = {'300474':[83,75.01,1],'000333':[86,81,1]}

alarm_dic = hl_dic.copy()
price_dict = hl_dic.copy()
for key in alarm_dic:
    alarm_dic[key] = 0
for key in price_dict:
    price_dict[key] = {'init':0,'win':[0,0,0,0,0],'loss':[0,0,0,0,0],'to_loss':[0,0,0,0,0],"min_los":0,'max_win':0}
    
while True:
    
    for key,value in hl_dic.items():
        print('------------------------------------------------------------------')
        # 获取价格
        try:
            df = ts.get_realtime_quotes(key) #Single stock symbol
        except:
            print('实时数据获取失败')
            break
        dt = df[['code','name','price','high','low','bid','ask','volume','amount','time']]
        current_price = dt.price[0]
        name = dt.name[0]
        time1 = dt.time[0]
        high = dt.high[0]
        low = dt.low[0]
        print(name + '  ' +key+' '+ '价格：' + str(current_price) + '  当日最高价: ' + str(high)+'  当日最低价: ' + str(low)+'  时间: ' + str(time1))
        
        # 首次买入提醒
        if value[2]==0:
            if float(current_price) >= value[1] and float(current_price) < value[1]*1.03 and float(low) < value[1]:
                engine = pyttsx3.init()
                if engine._inLoop:
                    engine.endLoop()
                engine = pyttsx3.init()
                engine.say(name+'已越过最低价,买入   '+'价格：' + str(current_price))
                engine.say(name+'已越过最低价,买入   '+'价格：' + str(current_price))
                engine.runAndWait()  
                print('首次买入声音信号已发送.................')
                time.sleep(5)
        
        else:
            # 构建所有止盈止损字典
            price_dict[key]['init'] = value[1]
            price_dict[key]['win'] = [round(value[1]*1.03,4),round(value[1]*1.05,4),round(value[1]*1.07,4),round(value[1]*1.09,4),round(value[1]*1.11,4)]
            price_dict[key]['loss'] = [round(value[1]*0.99,4),round(value[1]*1.02,4),round(value[1]*1.05,4),round(value[1]*1.08,4),round(value[1]*1.10,4)]
            price_dict[key]['min_los'] = value[1]*0.85
            price_dict[key]['max_win'] = 0.98*(value[0]+value[1])/2
            
            
            his_df = ts.get_hist_data(key, ktype='5')
            # 比较当天的价格位置
            if value[1]< float(low):
                de = his_df[(his_df.high>value[1]) & (his_df.low < value[1])]
                if len(de)!=0:
                    time1 = de.index[0] 
                    max_price = max(float(high),his_df[:time1].high.max())
                    print(time1,max_price)
                    if max_price >= price_dict[key]['win'][0] and max_price < price_dict[key]['win'][1]:
                        price_dict[key]['to_loss'][0] = 1
                    elif max_price >= price_dict[key]['win'][1] and max_price < price_dict[key]['win'][2]:
                        price_dict[key]['to_loss'][0] = 1
                        price_dict[key]['to_loss'][1] = 1
                    elif max_price >= price_dict[key]['win'][2] and max_price < price_dict[key]['win'][3]:
                        price_dict[key]['to_loss'][0] = 1
                        price_dict[key]['to_loss'][1] = 1
                        price_dict[key]['to_loss'][2] = 1
                    elif max_price >= price_dict[key]['win'][3] and max_price < price_dict[key]['win'][4]:
                        price_dict[key]['to_loss'][0] = 1
                        price_dict[key]['to_loss'][1] = 1
                        price_dict[key]['to_loss'][2] = 1
                        price_dict[key]['to_loss'][3] = 1    
                    elif max_price >= price_dict[key]['win'][4]:
                        price_dict[key]['to_loss'][0] = 1
                        price_dict[key]['to_loss'][1] = 1
                        price_dict[key]['to_loss'][2] = 1
                        price_dict[key]['to_loss'][3] = 1
                        price_dict[key]['to_loss'][4] = 1
            elif value[1] >= float(low) and value[1] <= float(high):
                if float(high) >= price_dict[key]['win'][0] and float(high) < price_dict[key]['win'][1]:
                        price_dict[key]['to_loss'][0] = 1
                elif float(high) >= price_dict[key]['win'][1] and float(high) < price_dict[key]['win'][2]:
                    price_dict[key]['to_loss'][0] = 1
                    price_dict[key]['to_loss'][1] = 1
                elif float(high) >= price_dict[key]['win'][2] and float(high) < price_dict[key]['win'][3]:
                    price_dict[key]['to_loss'][0] = 1
                    price_dict[key]['to_loss'][1] = 1
                    price_dict[key]['to_loss'][2] = 1
                elif float(high) >= price_dict[key]['win'][3] and float(high) < price_dict[key]['win'][4]:
                    price_dict[key]['to_loss'][0] = 1
                    price_dict[key]['to_loss'][1] = 1
                    price_dict[key]['to_loss'][2] = 1
                    price_dict[key]['to_loss'][3] = 1    
            elif value[1] > float(high):
                pass
#                 de = his_df[(his_df.high>value[1]) & (his_df.low < value[1])]
#                 if len(de)!=0:
#                     time = de.index[0] 
                
            print(price_dict)
        
            if float(current_price) > price_dict[key]['max_win']:
                if engine._inLoop:
                        engine.endLoop()
                engine = pyttsx3.init()
                engine.say(name+'终极止盈,卖出'+'价格：' + str(current_price))
                engine.say(name+'终极止盈,卖出'+'价格：' + str(current_price))
                engine.runAndWait()  
                print('终极止盈声音信号已发送.................')
                print('已到达终极止盈')
                time.sleep(5)
            else:
                if float(current_price) < price_dict[key]['min_los']:
                    if engine._inLoop:
                        engine.endLoop()
                    engine = pyttsx3.init()
                    engine.say(name+'终极止损,卖出'+'价格：' + str(current_price))
                    engine.say(name+'终极止损,卖出'+'价格：' + str(current_price))
                    engine.runAndWait()  
                    print('终极止损声音信号已发送.................')
                    print('已到达终极止损')
                    time.sleep(5)
                elif float(current_price) < price_dict[key]['loss'][0] and price_dict[key]['to_loss'][0]==1:
                    if engine._inLoop:
                        engine.endLoop()
                    engine = pyttsx3.init()
                    engine.say(name+'百分之3一级止盈,卖出'+'价格：' + str(current_price))
                    engine.say(name+'百分之3一级止盈,卖出'+'价格：' + str(current_price))
                    engine.runAndWait()  
                    print('百分之3一级止盈声音信号已发送.................')
                    print('已到达百分之3一级止盈')
                    time.sleep(5)
                elif float(current_price) < price_dict[key]['loss'][1] and price_dict[key]['to_loss'][1]==1:
                    if engine._inLoop:
                        engine.endLoop()
                    engine = pyttsx3.init()
                    engine.say(name+'百分之5二级止盈,卖出'+'价格：' + str(current_price))
                    engine.say(name+'百分之5二级止盈,卖出'+'价格：' + str(current_price))
                    engine.runAndWait()  
                    print('百分之5二级止盈声音信号已发送.................')
                    print('已到达百分之5二级止盈')
                    time.sleep(5)
                elif float(current_price) < price_dict[key]['loss'][2] and price_dict[key]['to_loss'][2]==1:
                    if engine._inLoop:
                        engine.endLoop()
                    engine = pyttsx3.init()
                    engine.say(name+'百分之7三级止盈,卖出'+'价格：' + str(current_price))
                    engine.say(name+'百分之7三级止盈,卖出'+'价格：' + str(current_price))
                    engine.runAndWait()  
                    print('百分之7三级止盈声音信号已发送.................')
                    print('已到达百分之7三级止盈')
                    time.sleep(5)
                elif float(current_price) < price_dict[key]['loss'][3] and price_dict[key]['to_loss'][3]==1:
                    if engine._inLoop:
                        engine.endLoop()
                    engine = pyttsx3.init()
                    engine.say(name+'百分之9四级止盈,卖出'+'价格：' + str(current_price))
                    engine.say(name+'百分之9四级止盈,卖出'+'价格：' + str(current_price))
                    engine.runAndWait()  
                    print('百分之9四级止盈声音信号已发送.................')
                    print('已到达百分之9四级止盈')
                    time.sleep(5)
                elif float(current_price) < price_dict[key]['loss'][4] and price_dict[key]['to_loss'][4]==1:
                    if engine._inLoop:
                        engine.endLoop()
                    engine = pyttsx3.init()
                    engine.say(name+'百分之11五级止盈,卖出'+'价格：' + str(current_price))
                    engine.say(name+'百分之11五级止盈,卖出'+'价格：' + str(current_price))
                    engine.runAndWait()  
                    print('百分之11五级止盈声音信号已发送.................')
                    print('已到达百分之11五级止盈')
                    time.sleep(5)
           
    time.sleep(5)

------------------------------------------------------------------
景嘉微  300474 价格：77.050  当日最高价: 77.450  当日最低价: 70.170  时间: 13:45:39
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
{'300474': {'init': 75.01, 'win': [77.2603, 78.7605, 80.2607, 81.7609, 83.2611], 'loss': [74.2599, 76.5102, 78.7605, 81.0108, 82.511], 'to_loss': [1, 0, 0, 0, 0], 'min_los': 63.758500000000005, 'max_win': 77.4249}, '000333': {'init': 0, 'win': [0, 0, 0, 0, 0], 'loss': [0, 0, 0, 0, 0], 'to_loss': [0, 0, 0, 0, 0], 'min_los': 0, 'max_win': 0}}
------------------------------------------------------------------
美的集团  000333 价格：83.000  当日最高价: 83.800  当日最低价: 81.360  时间: 13:45:42
本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
{'300474': {'init': 75.01, 'win': [77.2603, 78.7605, 80.2607, 81.7609, 83.2611], 'loss': [74.2599, 76.5102, 78.7605, 81.0108, 82.511], 'to_loss': [1, 0, 0, 0, 0], 'min_los': 63.758500000000005, 'max_win': 77.4249}, '000333': {'init': 81, 'win': [83.43, 85.05, 86.67, 88.29, 89.91], 'los

KeyboardInterrupt: 

In [51]:
import pandas as pd
from datetime import datetime
import os 
if 'log.pkl' not in os.listdir():
    df = pd.DataFrame([{'code':'000000','in':1,'time':datetime(2021,1,1,10,10,10),'price':0}])
    df.to_pickle("log.pkl")
else:
    df = pd.read_pickle("log.pkl")
    dict = [{'code':'000000','in':1,'time':datetime.now(),'price':0}]
    df1 = pd.DataFrame(dict)
    df = pd.concat([df,df1],axis=0)
    df = df.reset_index(drop=True)
    df.to_pickle("log.pkl")
    print(df)

     code  in                       time  price
0  000000   1 2021-01-01 10:10:10.000000      0
1  000000   1 2021-03-16 16:49:19.944095      0
2  000000   1 2021-03-16 16:49:22.689147      0
3  000000   1 2021-03-16 16:49:24.721899      0
4  000000   1 2021-03-16 16:49:26.498950      0
5  000000   1 2021-03-16 16:49:28.067215      0
6  000000   1 2021-03-16 16:49:30.992303      0
7  000000   1 2021-03-16 16:49:32.843228      0
8  000000   1 2021-03-16 16:49:34.771320      0


In [26]:
import pandas as pd
df = pd.read_pickle("log.pkl")
df

,level_0,index,in,time,price
0,0.0,0.0,1,2021-01-01 10:10:10.000000,0
1,1.0,0.0,1,2021-03-16 16:47:40.144266,0
2,0.0,NaN,1,2021-03-16 16:47:44.210982,0
3,NaN,NaN,1,2021-03-16 16:47:59.979523,0


In [16]:
import os
os.listdir()

['.ipynb_checkpoints',
 'log.pkl',
 'pyqtgraph.ipynb',
 'pyttsx3.ipynb',
 'Untitled Folder',
 '做T.ipynb',
 '数据爬取.ipynb']

In [60]:
hl_dic = {'601595':[17,12.88,1],'283283':[10,12,0]}
[i[-1] for i in list(hl_dic.values())]

[1, 0]

In [ ]:
 # 录入是否到达阶段最大值
            if high >= price_dict[key]['win'][0]:
                price_dict[key]['to_loss'][0] = 1
            elif high >= price_dict[key]['win'][1]:
                price_dict[key]['to_loss'][1] = 1:
            elif high >= price_dict[key]['win'][2]
                price_dict[key]['to_loss'][2] = 1:
            elif high >= price_dict[key]['win'][3]
                price_dict[key]['to_loss'][3] = 1:
            elif high >= price_dict[key]['win'][4]
                price_dict[key]['to_loss'][4] = 1:
            print(key,price_dict[key])
            # 判断提醒
            if float(current_price) > price_dict[key]['max_win']:
                if low < price_dict[key]['max_win']:
                    
                    if engine._inLoop:
                            engine.endLoop()
                    engine = pyttsx3.init()
                    engine.say(name+'终极止盈,卖出'+'价格：' + str(current_price))
                    engine.say(name+'终极止盈,卖出'+'价格：' + str(current_price))
                    engine.runAndWait()  
                    print('终极止盈声音信号已发送.................')
                print('已到达终极止盈')
            else:
                if float(current_price) < price_dict[key]['min_los'] and price_dict[key]['to_loss'][0]==0:
                    if pre_price > price_dict[key]['min_los']:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'终极止损,卖出'+'价格：' + str(current_price))
                        engine.say(name+'终极止损,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('终极止声音信号已发送.................')
                    print('已到达终极止损')
                elif float(current_price) < price_dict[key]['loss'][0] and price_dict[key]['to_loss'][0]==1:
                    if pre_price > price_dict[key]['loss'][0]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之3一级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之3一级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之3一级止盈声音信号已发送.................')
                    print('已到达百分之3一级止盈')
                elif float(current_price) < price_dict[key]['loss'][1] and price_dict[key]['to_loss'][1]==1:
                    if pre_price > price_dict[key]['loss'][1]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之5二级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之5二级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之5二级止盈声音信号已发送.................')
                    print('已到达百分之5二级止盈')
                elif float(current_price) < price_dict[key]['loss'][2] and price_dict[key]['to_loss'][2]==1:
                    if pre_price > price_dict[key]['loss'][2]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之7三级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之7三级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之7三级止盈声音信号已发送.................')
                    print('已到达百分之7三级止盈')
                elif float(current_price) < price_dict[key]['loss'][3] and price_dict[key]['to_loss'][3]==1:
                    if pre_price > price_dict[key]['loss'][3]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之9四级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之9四级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之9四级止盈声音信号已发送.................')
                    print('已到达百分之9四级止盈')
                elif float(current_price) < price_dict[key]['loss'][4] and price_dict[key]['to_loss'][4]==1:
                    if pre_price > price_dict[key]['loss'][4]:
                        if engine._inLoop:
                            engine.endLoop()
                        engine = pyttsx3.init()
                        engine.say(name+'百分之11五级止盈,卖出'+'价格：' + str(current_price))
                        engine.say(name+'百分之11五级止盈,卖出'+'价格：' + str(current_price))
                        engine.runAndWait()  
                        print('百分之11五级止盈声音信号已发送.................')
                    print('已到达百分之11五级止盈')
    '''